In [4]:
import pandas as pd
import os

data_dir = '../data' 

In [ ]:
# Récupère la liste des fichiers .csv
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(data_dir, file)
    
    df = pd.read_csv(file_path)
    
    original_len = len(df)
    
    # Supprime toutes les lignes de jours fériés (ex: 2015-01-01)
    df_clean = df.dropna(how='any')
    
    # Création du dossier processed s'il n'existe pas
    processed_dir = os.path.join(data_dir, 'processed')
    os.makedirs(processed_dir, exist_ok=True)
    
    output_file_path = os.path.join(processed_dir, file)
    df_clean.to_csv(output_file_path, index=False)
    
    print(f"Fichier traité : {file} - Lignes supprimées : {original_len - len(df_clean)}")

In [ ]:
all_data = []

for file in os.listdir(processed_dir):
    if file.endswith('.csv') and 'all_sectors' not in file:
        file_path = os.path.join(processed_dir, file)
        df = pd.read_csv(file_path)
        
        # Transformation en format Long (Date, Ticker, Price)
        # On suppose que la première colonne est 'Date' et les autres sont des Tickers
        df_melted = df.melt(id_vars=['Date'], var_name='Ticker', value_name='Close')
        
        # Ajout de la colonne Secteur
        df_melted['Sector'] = file.replace('.csv', '')
        
        all_data.append(df_melted)

# Concatenation de tous les dataframes
master_df = pd.concat(all_data, ignore_index=True)

# Conversion en datetime
master_df['Date'] = pd.to_datetime(master_df['Date'])

# Tri par Date et Ticker
master_df = master_df.sort_values(['Date', 'Ticker'])

# Sauvegarde du fichier concaténé
output_path = os.path.join(processed_dir, 'all_sectors.csv')
master_df.to_csv(output_path, index=False)

print(f"Fichier sauvegardé : {output_path}")
print(f"Nombre total de lignes : {len(master_df)}")
master_df.head()

Fichier sauvegardé : ../data/processed\all_sectors.csv
Nombre total de lignes : 417277


,Date,Ticker,Close,Sector
241532,2015-01-02,ABBV,42.091427,Health_Care
407213,2015-01-02,AEP,41.371307,Utilities
226436,2015-01-02,AIG,43.314121,Financials
371989,2015-01-02,ALB,51.527462,Materials
251596,2015-01-02,AMGN,116.436531,Health_Care
